# Collaborative Filtering Recommendar System

#### Using Implicit Data and Alternating Squares Recommender Model Fitting

# Clean Data

In [7]:
import pandas as pd
import numpy as np
import sys
import random
import string
import implicit

import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler

In [8]:
data = [   
    ('user_id', 'airport_id', 'airport', 'region', 'season'),
    # IF u355541 flew 8 times in the summer
        '''
        4 to asia    - then asian summer flights get a  4 - 0.2
        2 to florida - if florida destination is twice, then +0.2
        1 to europe  
        1 to latin
        '''
]

In [9]:
origs = [   
    (1, 'ORD', 'Midwest'),
    (2, 'LAX', 'West'),
    (3, 'SFO', 'West'),
    (4, 'IAH', 'South'),
    (5, 'DEN', 'Snow'),
    (6, 'IAD', 'East'),
    (7, 'EWR', 'East')
]

dests = [
    (1, 'ORD', 'Midwest'),
    (2, 'LAX', 'Sun'),
    (3, 'SFO', 'West'),
    (4, 'IAH', 'Sun'),
    (5, 'DEN', 'Snow'),
    (6, 'IAD', 'East'),
    (7, 'EWR', 'East'),
    (8, 'MCO', 'Sun'),
    (9, 'TPA', 'Sun'),
    (10, 'MIA', 'Sun'),
    (11, 'RSW', 'Sun'),

    # Europe
    (12, 'LHR', 'EURO'),
    (13, 'CVG', 'EURO'),
    (14, 'AMS', 'EURO'),
    (15, 'DUB', 'EURO'),
    (16, 'BRU', 'EURO'),
    (17, 'MUC', 'EURO'),
    (18, 'FRA', 'EURO'),
    (19, 'BHM', 'EURO'),

    # Asia
    (20, 'ICN', 'ASIA'),
    (21, 'NRT', 'ASIA'),
    (22, 'KIX', 'ASIA'),
    (23, 'PVG', 'ASIA'),
    (24, 'HKG', 'ASIA'),
    (25, 'PEK', 'ASIA'),
    (26, 'TPE', 'ASIA'),
    (27, 'SIN', 'ASIA'),

    # Latin America
    (28, 'EZE', 'LAT'),
    (29, 'GRU', 'LAT'),
    (30, 'GIG', 'LAT'),
    (31, 'BOG', 'LAT'),
    (32, 'SCL', 'LAT'),
    (33, 'TGU', 'LAT'),
    (34, 'MEX', 'LAT')
]

In [12]:
# Create an empty DataFrame
data = pd.DataFrame()

# Select amount of random data to generate 
n = 3000

# Append random data to new DataFrame columns
data['user_id'] = [random.choice(range(1,99)) for i in range(n)] 
data['origin_tuple'] = [random.choice(origs) for i in range(n)]
data['destination_tuple'] = [random.choice(dests) for i in range(n)]

# Transform Tuple() items to DataFrame column
data = data.join(pd.DataFrame(data['origin_tuple'].values.tolist(), columns = 
                              ['orig_id', 'orig', 'orig_reg']
                             )
                )
data = data.join(pd.DataFrame(data['destination_tuple'].values.tolist(), columns = 
                              ['dest_id', 'dest', 'dest_reg']
                             )
                )
data = data[[
    'user_id',
    'dest_id',
    'orig',
    'dest',
    'dest_reg'
]]

In [13]:
counts = data.groupby(['user_id', 'dest_reg'], as_index = False)['dest'].count()
counts = counts.rename(columns = {'dest': 'freq'})

data = pd.merge(data, counts, on = ['user_id', 'dest_reg'], how = 'left')

# Alternating Least Squares

In [14]:
'''
The implicit library expects data as a item-user matrix so we create two matricies,
one for fitting the model (item-user) and one for recommendations (user-item)
'''
sparse_item_user = sparse.csr_matrix((data['freq'].astype(float), (data['dest_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['freq'].astype(float), (data['user_id'], data['dest_id'])))

In [15]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors = 20, regularization = 0.1, iterations = 20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

In [16]:
# Find the n most similar items to airport_id
item_id = 23
n_similar = 10

# Use implicit to get similar items
similar = model.similar_items(item_id, n_similar)

# Print the names of our most similar airports
for item in similar:
    idx, score = item
    print(data.dest.loc[data.dest_id == idx].iloc[0])

PVG
KIX
MUC
BOG
MEX
GIG
GRU
TGU
MIA
SIN


In [18]:
# Create recommendations for user with id
user_id = int(input('Enter User ID: '))

# Use the implicit recommender
recommended = model.recommend(user_id, sparse_user_item)

airports = []
scores = []

# Get airport names from the ids
for item in recommended:
    idx, score = item
    airports.append(data.dest.loc[data.dest_id == idx].iloc[0])
    scores.append(score.round(3))

# Create a dataframe of airport names and scores
recommendations = pd.DataFrame({'Destination': airports, 'Score': scores})

recommendations[:5]

Enter User ID: 6


,Destination,Score
0,MUC,0.990
1,MEX,0.983
2,TPA,0.983
3,GIG,0.939
4,LAX,0.850
